In [103]:
%%writefile Final_Proj.py

import streamlit as st
from streamlit_option_menu import option_menu
import json
import requests
import pandas as pd
from geopy.distance import geodesic
import statistics
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

# ----------------------------------------Functions---------------------------------------------------------
# To get mrt_df
def get_mrt_df():
    df = pd.read_csv('mrt.csv').reset_index(drop= True)
    return df

# To load pickles
def load_pickles():
    with open(r"model.pkl", 'rb') as file:
        loaded_model = pickle.load(file)
    with open(r'scaler.pkl', 'rb') as f:
        scaler_loaded = pickle.load(f)
    with open(r'ohe.pkl', 'rb') as f:
        ohe_loaded = pickle.load(f)
    return loaded_model, scaler_loaded, ohe_loaded





             # Pre_processing input    
def preprocessing_input(street_name,block,lease_commence_year,storey_range):
    # -----Calculating lease_remain_years using lease_commence_year-----
    lease_remain_years = 99 - (2023 - lease_commence_year)

    # -----Calculating median of storey_range to make our calculations quite comfortable-----
    split_list = storey_range.split(' TO ')
    float_list = [float(i) for i in split_list]
    storey_median = statistics.median(float_list)
    min_dist_mrt, cbd_dist = get_min_distance_from_MRT_CBD(block,street_name)
    
    return storey_median,lease_remain_years, min_dist_mrt, cbd_dist

# -------------------------------This is the configuration page for our Streamlit Application---------------------------
st.set_page_config(
    page_title="Bank Credit Risk Control",
    page_icon="🏦",
    layout="wide"
)

# -------------------------------This is the sidebar in a Streamlit application, helps in navigation--------------------
with st.sidebar:
    selected = option_menu("Main Menu", ["About Project", "Predictions","EDA Visualiztion","Movie Review Sentiment Analysis"],
                           icons=["house", "gear","list-task","cloud-upload"],
                           styles={"nav-link": {"font": "sans serif", "font-size": "20px", "text-align": "centre"},
                                   "nav-link-selected": {"font": "sans serif", "background-color": "#0072b1"},
                                   "icon": {"font-size": "20px"}
                                   }
                           )

# -----------------------------------------------About Project Section--------------------------------------------------
if selected == "About Project":
    st.markdown("# :blue[Credit Risk Analysis]")
    st.markdown('<div style="height: 50px;"></div>', unsafe_allow_html=True)
    st.markdown("### :blue[Technologies :] Python, Pandas, Numpy, Scikit-Learn, Streamlit, Seaborn, Matplotlib "
                "Machine Learning, Data Preprocessing, Visualization, EDA, Model Building, Data Wrangling, "
                "Model Deployment")
    st.markdown("### :blue[Overview :] This project aims to construct a machine learning model and implement "
                "it as a bank credit risk  application in order to provide accurate predictions whether the "
                "client would repay the loan or not. This prediction model will be based on past transactions "
                "involving previous loan applications, and its goal is to accurately predict whether the client can repay "
                "loan. The criteria are plentlifull when it comes to banking data which includes, Whether client has a property"
                "of criteria, including location, the kind of apartment, the total square footage, and the length "
                "or Car on their own, their rating of their locality and reliable ratings score based on client's credit acitivity"
                "This predictive model will assist whether the client is reliable to pay off the loan")
    st.markdown("### :blue[Domain :] Banking")

# ------------------------------------------------Predictions Section---------------------------------------------------
if selected == "Predictions":
    st.markdown("# :blue[Predicting Results based on Trained Models]")
    st.markdown("### :orange[Predicting Whether the client will repay the loan]")
    
    
    with st.form("form1"):
    
    # -----New Data inputs from the user for predicting the resale price-----
        st.write( f'<h5 style="color:rgb(0, 153, 153,0.4);">NOTE: Min & Max given for reference, you can enter any value</h5>', unsafe_allow_html=True )
        col1,col2 = st.columns(2)
        
        with col1:
            loan_id = st.text_input("Loan ID")
            age = st.number_input("Age")
            flag_car = st.radio("Does client own a Car?",['Yes','No'], key = 'car', horizontal=True)
            gender = st.radio("Gender", ['Male','Female'], key = 'Gender', horizontal= True)        
            amt_req_credit_mon = st.number_input("Number of enquiries to Credit Bureau about the client one month before application")   
            total_income = st.number_input('Total Income')
            days_registration = st.number_input("How many days before the application did client change his registration",)
            amt_annuity = st.number_input("Loan Annuity")

        with col2:
            name = st.text_input("Name")
            region_rating = st.slider(label = 'Region_Rating', min_value= 1, max_value= 3)
            flag_prop = st.radio("Does customer own a Property?",['Yes','No'], key = 'prop', horizontal= True)
            experience = st.number_input('Work Experience (In Years)', step = 1)
            def_60_cnt_circle = st.number_input("How many observation of client's social surroundings defaulted on 30 DPD (days past due) ")
            total_documents = st.number_input("Total Documents submitted by client", step = 1)
            amt_credit = st.number_input("Loan Credit Amount")
            exit_score = st.number_input("Exit Score by 3rd party")
            st.markdown("""
                    <style>
                    div.stButton > button:first-child {
                        background-color: #009999;
                        color: white;
                        width: 100%;
                    }
                    </style>
                """, unsafe_allow_html=True)
        # -----Submit Button for PREDICT RESALE PRICE-----

        submit_button = st.form_submit_button(label="PREDICT STATUS")
        if gender is 'Female':
            gender = 0
        else:
            gender = 1
            
        if flag_car is 'No':
            flag_car = 0
        else:
            flag_car = 1
            
        if flag_prop is 'No':
            flag_prop = 0
        else:
            flag_prop = 1
            
        if total_income <= 112500.0:
            income_range = 'Low'
        elif total_income > 112500.0 and total_income <= 157500.0:
            income_range = 'Medium'
        elif total_income > 157500.0 and total_income <= 202500.0:
            income_range = 'High'
        elif total_income > 202500.0:
            income_range = 'Very High'
        
        if age <= 33:
            age_category = '21-33'
        elif age > 33 and age <= 45:
            age_category = '34-45'
        elif age > 45 and age <= 57:
            age_category = '46-57'
        elif age > 57:
            age_category = '58-69'

        if experience <= 5:
            employment_year = '0-5Y'
        elif experience > 5 and experience <= 10:
            employment_year = '5-10Y'
        elif experience > 10 and experience <= 20:
            employment_year = '10-20Y'
        elif experience > 20 and experience <= 30:
            employment_year = '20-30Y'
        elif experience > 30 and experience <= 40:
            employment_year = '30-40Y'
        elif experience > 40 and experience <= 50:
            employment_year = '40-50Y'
        elif experience > 50:
            employment_year = '50 above'
        
        
     
       
       
        if submit_button is True:
            try: 
                model_loaded, scaler_loaded ,ohe_loaded = load_pickles()
                #X = [gender, flag_prop, employment_year, total_income, total_documents, days_registration, def_60_cnt_circle,
                #amt_req_credit_mon, amt_annuity, exit_score, flag_car, region_rating, amt_credit,age_category,income_range]
                X_ohe = ohe_loaded.transform([[employment_year, age_category,income_range]]).toarray()
                X_scaled = scaler_loaded.transform([[total_income,days_registration,amt_annuity,exit_score,amt_credit]])
                X_scaled = np.array(X_scaled)
                X_num = np.array([[gender, flag_prop,total_documents,def_60_cnt_circle,amt_req_credit_mon,flag_car,region_rating]])
                #np.concatenate((X_num,X_scaled),axis = 1)
                X= np.concatenate([X_num,X_scaled,X_ohe],axis = 1)
                pred = model_loaded.predict(X)
                if(pred == 0):
                    st.markdown("# :green[Client would repay the loan amount]")
                else:
                    st.markdown("# :red[Client would not repay the loan amount]")
            except Exception as err:
                st.warning("Please fill the form with Valid Details")
                pass
elif selected == "EDA Visualiztion":
    tab1, tab2, tab3, = st.tabs(["***MODEL SELECTION***","***EDA VISUALIZATION***","***INSIGHTS***"])

    with tab1:
         st.header("Model Performance")
         data = {
                    "Algorithm": ["Decision Tree","Random Forest","KNN","XGradientBoost"],
                    "Accuracy": [89,88,97,93],
                    "Precision": [90,90,96,94],
                    "Recall": [89,89,96,94],
                    "F1 Score": [89,89,97,94]
                    
                    }
         dff = pd.DataFrame(data)
         st.dataframe(dff)
         st.markdown(f"## The Selected Algorithm is :orange[*XGradient Boost*] and its Accuracy is   :orange[*93%*]")

    
elif selected == "Movie Review Sentiment Analysis":
    review_txt = st.text_area('Write a Review about movie')
    review_txt = review_txt.lower()
    arr = []
    arr.append(review_txt)
    #review_txt = list(review_txt.split(" "))

    movie_submit =  st.button("Check Sentiment")
    if movie_submit is True:
        
        with open(r"countVect.pkl", 'rb') as file:  
            countVect = pickle.load(file)
        with open(r"logisticRegression.pkl",'rb') as file:
            logisticRegression = pickle.load(file)
            
        testFeatures = countVect.transform(arr)
        features = countVect.get_feature_names_out()
        testPredictLR = logisticRegression.predict(testFeatures)
        if testPredictLR is 1:
            st.markdown("# :green[The movie got a positive review]")
        else:
            st.markdown("# :red[The movie got a negative review]")
            
            
            

        
    




Overwriting Final_Proj.py
